In [9]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation

from models import lungModel

In [20]:
def create_pgmpy_CPT(cpt, A_label, B_label=''):
    if B_label == '':
        return TabularCPD(variable=A_label,
                          variable_card=len(cpt),
                          values=[[cpt[key]] for key in cpt.keys()])
    else:
        return TabularCPD(variable=A_label,
                          variable_card=len(cpt),
                          values=[list(cpt[key].values()) for key in cpt.keys()],
                          evidence=[B_label],
                          evidence_card=[len(cpt[list(cpt.keys())[0]])])

In [25]:
# Model definition
m = lungModel()

# Add graph
lung_model = BayesianNetwork([('Inflammation', 'Wellness'), ('Inflammation', 'Bacterial load'), ('Bacterial load', 'FEV1')])

# Define the CPDs
cpd_i = create_pgmpy_CPT(m.marginal_i, 'Inflammation')
cpd_w = create_pgmpy_CPT(m.cpt_w_i, 'Wellness', 'Inflammation')
cpd_bl = create_pgmpy_CPT(m.cpt_bl_i, 'Bacterial load', 'Inflammation')
cpd_FEV1 = create_pgmpy_CPT(m.cpt_FEV1_bl, 'FEV1', 'Bacterial load')

# Add the CPDs to the model
lung_model.add_cpds(cpd_i, cpd_w, cpd_bl, cpd_FEV1)

# Checks
# Checking if the CPDs are valid for the model
print("\nThe CPTs are valid:", lung_model.check_model())
# List of independence conditions for Bacterial load
print("\nIndependence conditions for Bacterial load:", lung_model.local_independencies('Bacterial load'))
# Get all model independence conditions
# print("\nIndependence conditions across the whole model:\n", lung_model.get_independencies())


The CPDs are valid: True

Independence conditions for Bacterial load: (Bacterial load ⟂ Wellness | Inflammation)


In [26]:
# CASE 1
belief_propagation = BeliefPropagation(lung_model)
final_dist = belief_propagation.query(variables=['Inflammation'], evidence={'Bacterial load': 1})
print(final_dist.values)

# CASE 2
belief_propagation = BeliefPropagation(lung_model)
final_dist = belief_propagation.query(variables=['Inflammation'], evidence={'FEV1': 0, 'Wellness': 2})
print(final_dist.values)

# CASE 3
belief_propagation = BeliefPropagation(lung_model)
final_dist = belief_propagation.query(variables=['Bacterial load'], evidence={'Inflammation': 1})
print(final_dist.values)

# CASE 4
belief_propagation = BeliefPropagation(lung_model)
final_dist = belief_propagation.query(variables=['FEV1'], evidence={'Wellness': 0})
print(final_dist.values)

  0%|          | 0/2 [00:00<?, ?it/s]

[0.22222222 0.55555556 0.22222222]


  0%|          | 0/1 [00:00<?, ?it/s]

[0.33333333 0.58712121 0.07954545]


  0%|          | 0/2 [00:00<?, ?it/s]

[0.3 0.3 0.4]


  0%|          | 0/2 [00:00<?, ?it/s]

[0.2684153  0.37825137 0.35333333]
